# Homework 3

* Assigned: 10/10

* Due: 10/29 @ 11:59PM ET on Gradescope

* Value: 3.75% of your grade

In this assignment it's time to get real! You'll first flex your SQL muscles and perform analyses similar to HW2's by writing SQL and reflecting on the experience. You will then perform some normalization.

# Assignment description

In this assignment, we will explore how to train a decision tree model over joins using SQL and implement some core functions in training the model. We will break down the steps into smaller problems. 

Each decision tree can be understood as a set of (`condition`, `prediction`) pairs that assign a `prediction` to rows in the dataset satisfying the `condition`. To train a decision tree model, we learn the `condition` by iteratively splitting the dataset. In this homework, we consider the equal/not equal split based on a `value` for each `attribute`. That is, each `condition` will be of the form `attribute == value` and `attribute != value`. 

For the split criteria, we will consider minimizing variance for each split. To compute variance of an attribute, we use the formula $Var(X) = \frac{1}{n}\sum (x_i - \bar{x})^2$. For the target variable $Y$, we would like to find the best (`attribute`, `value`) pair such that the $Var(Y)$ subtracting the sum of $Var(Y)$ in the dataset satisfying `attribute == value` and `attribute != value` is maximized.

We will also be using [DuckDB](https://duckdb.org/), a new database system designed for analytics.  It is very similar to the SQLite database we have used in the past, however it is _must faster_ when analyzing the entire dataset.

# Setup

We will use 2 tables from the favorita database (out of the 7 tables): `sales` is a fact table containing sales facts. `items` is a dimension table where the `item_nbr` in `sales` refer to that in `items`. Therefore, joining `sales` with `items` will lead to a table with the same number of tuples as that in `sales`.

We would like to write the core functions that trains a decision tree model to predict the `unit_sales` attribute in the `sales` table using the `family` attribute in `items`. 

In [2]:
!pip install duckdb

  Using cached duckdb-1.1.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (762 bytes)
Using cached duckdb-1.1.2-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (20.1 MB)


In [45]:
import duckdb

con = duckdb.connect(database=':memory:')
con.execute("CREATE OR REPLACE TABLE items AS SELECT * FROM 'items.csv';")
con.execute("CREATE OR REPLACE TABLE sales AS SELECT * FROM 'sales.csv';")

def runq(q):
    "I'm helper function to run and print queries"
    cursor = con.execute(q)
    df = cursor.fetchdf()
    return df

# Question 1

We will first discuss the naive approach where we materialize the join and compute the variance over the target variable for a potential split. We will consider the `family` attribute in `items` as an example. For each split $\sigma_{family=value}$ and $\sigma_{family\ne value}$, we will need to compute the variance of the target variable.

## 1.1
To compute the variance, one important statistics we need is the mean of `unit_sales` for $\sigma_{family=value}$ and $\sigma_{family\ne value}$ for each $value$ in the domain of `items.family`. Write a query to compute the mean of `unit_sales` for $\sigma_{family=value}$ and $\sigma_{family\ne value}$ for each `family`.

In [ ]:
q11 = """
create or replace table mat_suff_stat as
with total as (
    select 
        count(unit_sales) as TC,
        sum(unit_sales) as TS,
        sum(unit_sales * unit_sales) as TQ
    from sales join items on sales.item_nbr = items.item_nbr
),
family_stats as (
    select family, sum(unit_sales) as s, count(unit_sales) as c, sum(unit_sales * unit_sales) as Q
    from sales join items on sales.item_nbr = items.item_nbr
    group by family
)
select family, s, c, Q, TC, TS, TQ
from total, family_stats;

select family, 
s/c as mean_equal, 
(TS - s)/(TC - c) as mean_nequal
from mat_suff_stat
"""
runq(q11)

## 1.2
To compute the best split, we need to compute the maximum reduction in variance for each potential split point ($\sigma_{family=value}$ and $\sigma_{family\ne value}$). Modify your query in 1.1 to compute the sum of variance of `unit_sales` in $\sigma_{family=value}$ and $\sigma_{family\ne value}$.

In [ ]:
q12 = """
create or replace table var as
select family, 
(Q - s*s/c)/c as var_equal, 
(TQ-Q - (TS - s)*(TS - s)/(TC - c))/(TC - c) as var_nequal
from mat_suff_stat;

select * from var;
"""
runq(q12)

## 1.3
Next, compute the maximum reduction in variance and select the best split point. i.e. the value of `family`.

In [62]:
q13 = """
select family, var_equal + var_nequal as var_sum
from var
order by var_sum asc
limit 1;
"""
runq(q13)

,family,var_sum
0,HOME APPLIANCES,375.469571


# Question 2

Note that the approach above requires materializing the join. However, `item` is way smaller than `sales` so if we naively join the two tables together, rows in the `item` table will be duplicated. To do better, we may exploit a technique called $\textit{factorized learning}$. In Q4 from the last homework, we see that for two tables `R` and `S` we may pre-aggregate the sum of `X` in `R` group by `A` and the count in `S` group by `A`, and use them to compute the sum of `R.X` in the join in $R \Join_A S$. Here, we will use a similar technique to avoid materializing the join.

Based on the fact that $Var(X) = \frac{1}{n}\sum (x_i - \bar{x})^2 = \frac{1}{n} \left(\sum x_i^2 - \bar{x}\sum x_i\right)$ for a random variable $X$, we need to compute $Var(sales)$ when `family` is equal to some value and `family` not equal to that value ($\bar{x}$ is mean of `sales` in both cases, respectively). Following the strategies in HW2 Q4, we may apply pre-aggregating by breaking down $Var(X)$ into $\sum x_i^2 - \bar{x}\sum x_i$ for each `family`.

## 2.1
Write a query to pre-aggregate `sales` and `item` to compute the count of all rows in `sales` join `items` for each `family` and `item_nbr` pair. You will create (1) a table containing `item_nbr` and another column for `sales`, (2) a table containing `item_nbr`, `family`, and another column for `items`, and (3) your select statement.

In [68]:
q21 = """
create or replace table sales_c as
select item_nbr, count(unit_sales) as c
from sales
group by item_nbr;

create or replace table items_c as
select item_nbr, family, count(*) as c
from items
group by item_nbr, family;

select family, items_c.item_nbr, sum(sales_c.c * items_c.c) as c
from sales_c join items_c
on sales_c.item_nbr = items_c.item_nbr
group by family, items_c.item_nbr
"""
runq(q21)

,family,item_nbr,c
0,GROCERY I,105575,150.0
1,GROCERY I,105737,128.0
2,GROCERY I,108634,56.0
3,GROCERY I,108797,297.0
4,GROCERY I,114790,328.0
...,...,...,...
3993,BEVERAGES,1464196,77.0
3994,GROCERY I,1159413,311.0
3995,GROCERY I,1363877,34.0
3996,DELI,732007,271.0


## 2.2
Write a query to pre-aggregate `sales` and `item` to compute the sum of `unit_sales` in `sales` join `items` for each `family` and `item_nbr` pair. You will create (1) a table containing `item_nbr` and another column for `sales`, (2) a table containing `item_nbr`, `family`, and another column for `items`, and (3) your select statement. (The extra columns you created in this question can be the same as that in 2.1)

In [71]:
q22 = """
create or replace table sales_s as
select item_nbr, sum(unit_sales) as s
from sales
group by item_nbr;

create or replace table items_c as
select item_nbr, family, count(*) as c
from items
group by item_nbr, family;

select family, items_c.item_nbr, sum(sales_s.s * items_c.c) as s
from sales_s join items_c 
on sales_s.item_nbr = items_c.item_nbr
group by family, items_c.item_nbr
"""
runq(q22)

,family,item_nbr,s
0,GROCERY I,105575,3127.0
1,GROCERY I,105737,933.0
2,GROCERY I,108634,238.0
3,GROCERY I,108797,1562.0
4,GROCERY I,114790,4108.0
...,...,...,...
3993,BEVERAGES,1464196,228.0
3994,GROCERY I,1159413,3656.0
3995,GROCERY I,1363877,147.0
3996,DELI,732007,1012.0


## 2.3
Write a query to pre-aggregate `sales` and `item` to compute the sum of square of `unit_sales` in `sales` join `items` for each `family` and `item_nbr` pair. You will create (1) a table containing `item_nbr` and another column for `sales`, (2) a table containing `item_nbr`, `family`, and another column for `items`, and (3) your select statement (The extra columns you created in this question can be the same as that in 2.1 and 2.2)

In [72]:
q23 = """
create or replace table sales_Q as
select item_nbr, sum(unit_sales * unit_sales) as Q
from sales
group by item_nbr;

create or replace table items_c as
select item_nbr, family, count(*) as c
from items
group by item_nbr, family;

select family, items_c.item_nbr, sum(sales_Q.Q * items_c.c) as Q
from sales_Q join items_c 
on sales_Q.item_nbr = items_c.item_nbr
group by family, items_c.item_nbr
"""
runq(q23)

,family,item_nbr,Q
0,GROCERY I,103520,5.943000e+03
1,GROCERY I,105857,2.877900e+04
2,DELI,108698,4.669000e+03
3,CLEANING,108786,5.413700e+04
4,EGGS,108833,1.712000e+03
...,...,...,...
3993,BEVERAGES,1958249,4.368000e+03
3994,GROCERY I,2010756,2.920000e+03
3995,GROCERY I,1093344,5.500000e+02
3996,GROCERY I,1946668,5.000000e+01


## 2.4
Combining your solutions for 2.1, 2.2 and 2.3, you may pre-aggregate `sales` and `item` and only use the pre-aggregations to compute (write a query) the count of rows, sum of `unit_sales`, and sum of square of `unit_sales` for each `family`.

In [75]:
q24 = """
create or replace table sales_agg as
select item_nbr, count(unit_sales) as c, sum(unit_sales) as s, sum(unit_sales * unit_sales) as Q
from sales
group by item_nbr;

create or replace table family_agg as
select 
    family, 
    sum(sales_agg.c * items_c.c) as c,  
    sum(sales_agg.s * items_c.c) as s, 
    sum(sales_agg.Q * items_c.c) as Q
from sales_agg join items_c 
on sales_agg.item_nbr = items_c.item_nbr
group by family;

select * from family_agg;
"""
runq(q24)

,family,c,s,Q
0,POULTRY,6881.0,1.263237e+05,1.476727e+07
1,BREAD/BAKERY,18653.0,1.701709e+05,4.724414e+06
2,FROZEN FOODS,6278.0,5.751414e+04,1.059257e+07
3,PREPARED FOODS,3039.0,3.443715e+04,1.034652e+06
4,MEATS,9678.0,1.191989e+05,7.747543e+06
5,SEAFOOD,1079.0,8.616261e+03,1.632837e+05
6,"LIQUOR,WINE,BEER",4164.0,2.944700e+04,1.164091e+06
7,HARDWARE,270.0,4.100000e+02,8.720000e+02
8,HOME CARE,11368.0,6.372500e+04,9.748570e+05
9,LADIESWEAR,1082.0,2.545000e+03,1.018100e+04


## 2.5
Using your solution 2.4, write a query to compute variance of `unit_sales` for each `family`.

In [77]:
q25 = """
select family, (Q - s*s/c)/c as var_equal, 
from family_agg
"""
runq(q25)

,family,var_equal
0,POULTRY,1809.064853
1,BREAD/BAKERY,170.050343
2,FROZEN FOODS,1603.324480
3,PREPARED FOODS,212.049797
4,MEATS,648.835750
5,SEAFOOD,87.561902
6,"LIQUOR,WINE,BEER",229.550319
7,HARDWARE,0.923731
8,HOME CARE,54.331203
9,LADIESWEAR,3.876932


## 2.6
Similarly, using your solution 2.4, write a query to compute variance of the `unit_sales` whose `family` values does not equal to each `family` values.

In [85]:
q26 = """
with agg_total as (
    select 
        sum(c) as TC,
        sum(s) as TS,
        sum(Q) as TQ
    from family_agg
)

select 
    family, 
    (TQ - Q - ((TS - s) * (TS - s))/(TC - c))/(TC - c) as var_nequal
from family_agg, agg_total;
"""
runq(q26)

,family,var_nequal
0,POULTRY,353.116800
1,BREAD/BAKERY,382.403662
2,FROZEN FOODS,358.863927
3,PREPARED FOODS,375.440024
4,MEATS,368.794596
5,SEAFOOD,375.113843
6,"LIQUOR,WINE,BEER",375.692556
7,HARDWARE,374.669068
8,HOME CARE,381.735193
9,LADIESWEAR,375.214086


In [86]:
qcheck = """
with agg_total as (
    select 
        sum(c) as TC,
        sum(s) as TS,
        sum(Q) as TQ
    from family_agg
)

select 
    family, 
    (Q - s*s/c)/c + (TQ - Q - ((TS - s) * (TS - s))/(TC - c))/(TC - c) as sum_var
from family_agg, agg_total
order by sum_var asc
limit 1;
"""
runq(qcheck)

,family,sum_var
0,HOME APPLIANCES,375.469571


### Bonus
`Python Pros:`

Python can perform more complicated logic on data, for example we can implement algorithms in an imperative 
programming language such as Python but not in SQL.

With python, we can achieve complex data visualizations over web applications, which cannot be done through SQL.

`Python Cons:`

Python syntax on data manipulate is obscure sometimes.

Python data manipulate is hard to recover and rollback.


`SQL Pros:`

SQL is specialized to data transformations.

SQL is safer and more efficient because DBMS knows how to analyze it.

SQL provides ACID properties for transactions in database.

`SQL Cons`:

Require cross-language data pipeline for brining SQL analysis into product.